In [ ]:
# CRT Veto using CRT Space Points

In [ ]:
import ROOT
import numpy as np
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import sys
import multiprocessing as mp
import uproot
import pandas as pd
import pickle
import h5py
import gzip
import math
import timeit

# First attempt
#infile = "../NTuples/eta_production_ntuple_test_nodirt.root"

# Latest Eta Production
#infile = "../NTuples/eta_production_nodirt_ntuple_batch1.root"

# MC CV Sample --> Low Stats
#infile = "../NTuples/mc_production_with_fmatch_ntuple.root"

#MC CV Sample --> High Stats
#infile = "../NTuples/mc_production_with_fmatch_ntuple_more_stats.root"
infile = "../NTuples/mc_production_with_fmatch_ntuple_more_stats_v2.root"
crtfile = "../NTuples/mc_production_crt_ntuple_v1.root"

In [ ]:
inFile = uproot.open(infile)
crtFile = uproot.open(crtfile)

inFileROOT = ROOT.TFile.Open(infile, "READ")
#h_tot_pot = inFileROOT.Get("TotalPOT")
h_tot_pot = inFileROOT.Get("TOTPOT_Clone")
TOT_POT = h_tot_pot.GetBinContent(1)
inFileROOT.Close()
TOT_POT = f"{TOT_POT:.2e}"
print("Total POT", TOT_POT)

#slc_tree = inFile["slc_truth_tree"]
slc_tree = inFile["slc_truth_tree"]
slc_reco_tree = inFile["slc_tree"]
pfp_tree = inFile["pfp_tree"]

trk_tree1 = inFile["track_tree1"]
trk_tree2 = inFile["track_tree2"]

#cosmic_tree1 = inFile["cosmic_tree1"]
#cosmic_tree2 = inFile["cosmic_tree2"]

particle_tree1 = inFile["particle_tree1"]
particle_tree2 = inFile["particle_tree2"]

track_crt_tree = inFile["track_crthit_tree"]


pfp_df = pfp_tree.arrays(pfp_tree.keys(), library="pd")

slc_df = slc_tree.arrays(slc_tree.keys(), library="pd")
slc_reco_df = slc_reco_tree.arrays(slc_reco_tree.keys(), library="pd")

trk_df1 = trk_tree1.arrays(trk_tree1.keys(), library="pd")
trk_df2 = trk_tree2.arrays(trk_tree2.keys(), library="pd")

particle_df1 = particle_tree1.arrays(particle_tree1.keys(), library="pd")
particle_df2 = particle_tree2.arrays(particle_tree2.keys(), library="pd")

track_crt_df = track_crt_tree.arrays(track_crt_tree.keys(), library="pd")

#cosmic_df1 = cosmic_tree1.arrays(cosmic_tree1.keys(), library="pd")
#cosmic_df2 = cosmic_tree2.arrays(cosmic_tree2.keys(), library="pd")

crt_tree = crtFile["crt_tree"]

crt_df = crt_tree.arrays(crt_tree.keys(), library="pd")
crt_df = crt_df.drop_duplicates()

slc_df[:2]

In [ ]:
def isTPC(row):
    if (-200 <= row["vtx_x"] <= 200) and (-200 <= row["vtx_y"] <= 200) and (0 <= row["vtx_z"] <= 500):
        return 1
    else:
        return 0

slc_df["inTPC"] = slc_df.apply(isTPC, axis=1)
slc_reco_df["inTPC"] = slc_reco_df.apply(isTPC, axis=1)
slc_df[:2]

In [ ]:
topology_labels = {
    0:r"$\nu_{\mu} CC$",
    1:r"$\nu_{\mu} NC$",
    2:r"$\nu_{e} CC$",
    3:r"$\nu_{e} NC$",
    4:r"$\bar{\nu}_{\mu}$",
    5:r"$\bar{\nu}_{e}$",
    6:r"DIRT $\nu$",
    7:"Cosmic",
}

topology_selections = {
    0:"pdg == 14.0 and iscc == 1.0 and inTPC == 1",
    1:"pdg == 14.0 and isnc == 1.0 and inTPC == 1",
    2:"pdg == 12.0 and iscc == 1.0 and inTPC == 1",
    3:"pdg == 12.0 and isnc == 1.0 and inTPC == 1",
    4:"pdg == -14.0 and inTPC == 1",
    5:"pdg == -12.0 and inTPC == 1",
    6:"(pdg == 14.0 or pdg == 12.0 or pdg == -14.0 or pdg == -12.0 ) and inTPC == 0",
    7:"pdg == -1",
}

In [ ]:
for num in range(len(topology_selections.keys())):
    #temp = slc_df.query(topology_selections[num])
    condition = slc_df.index.isin(slc_df.query(topology_selections[num]).index)
    slc_df.loc[condition, "TOP"] = num

slc_reco_df["TOP"] = slc_df["TOP"]
slc_reco_df[:2]

In [ ]:
# Add topology labels to other dataframes
df_small_filtered = slc_df[['run', 'subrun', 'evt', 'slc', 'TOP']]

pfp_df = pfp_df.merge(df_small_filtered, on=['run', 'subrun', 'evt', 'slc'], how='left')
#cosmic_df1 = cosmic_df1.merge(df_small_filtered, on=['run', 'subrun', 'evt', 'slc'], how='left')
particle_df1 = particle_df1.merge(df_small_filtered, on=['run', 'subrun', 'evt', 'slc'], how='left')
track_crt_df = track_crt_df.merge(df_small_filtered, on=['run', 'subrun', 'evt', 'slc'], how='left')
trk_df1 = trk_df1.merge(df_small_filtered, on=['run', 'subrun', 'evt', 'slc'], how='left')
trk_df2["TOP"] = trk_df1["TOP"]
slc_reco_df["TOP"] = slc_df["TOP"]
particle_df1[:2]

# Initialize the Event Dataframe

In [ ]:
runs = slc_df["run"].values
subruns = slc_df["subrun"].values
evts = slc_df["evt"].values

data = {'run': runs,
        'subrun': subruns,
        'evt': evts}

event_df = pd.DataFrame(data)

event_df = event_df.drop_duplicates()

# Drop duplicates to find unique combinations
#unique_combinations = event_df.drop_duplicates()

# Get the number of unique combinations
N_EVENTS = event_df.shape[0]

print("N_EVENTS:", N_EVENTS)


In [ ]:
# Define Code to handle Neutrino stuff

In [ ]:
track_crt_df[:4]

# Select the AV and DIRT Rows

In [ ]:
# Select AV and DIRT

av_sel = "TOP < 5.8"
dirt_sel = "5.8 < TOP < 6.2"

slc_av_df = slc_df.query(av_sel)
slc_dirt_df = slc_df.query(dirt_sel)

slc_av_reco_df = slc_reco_df.query(av_sel)
slc_dirt_reco_df = slc_reco_df.query(dirt_sel)

pfp_av_df = pfp_df.query(av_sel)
pfp_dirt_df = pfp_df.query(dirt_sel)

track_crt_av_df = track_crt_df.query(av_sel)
track_crt_dirt_df = track_crt_df.query(dirt_sel)

trk_av_df1 = trk_df1.query(av_sel)
trk_av_df2 = trk_df2.query(av_sel)

trk_dirt_df1 = trk_df1.query(dirt_sel)
trk_dirt_df2 = trk_df2.query(dirt_sel)

print("Selected slices of interest")

In [ ]:
trk_dirt_df1[:5]

In [ ]:
# Strange Check

N_av_slc_reco_rows = slc_av_reco_df.shape[0]
test_df = slc_av_reco_df.drop_duplicates()
N_test = test_df.shape[0]

print(N_av_slc_reco_rows)
print(N_test)

In [ ]:
# I guess we need to drop some duplicates ??

slc_av_df = slc_av_df.drop_duplicates()
slc_av_reco_df = slc_av_reco_df.drop_duplicates()

slc_dirt_df = slc_dirt_df.drop_duplicates()
slc_dirt_reco_df = slc_dirt_reco_df.drop_duplicates()

pfp_av_df = pfp_av_df.drop_duplicates()
pfp_dirt_df = pfp_dirt_df.drop_duplicates()

track_crt_av_df = track_crt_av_df.drop_duplicates()
track_crt_dirt_df = track_crt_dirt_df.drop_duplicates()

trk_av_df1 = trk_av_df1.drop_duplicates()
trk_av_df2 = trk_av_df2.drop_duplicates()
trk_dirt_df1 = trk_dirt_df1.drop_duplicates()
trk_dirt_df2 = trk_dirt_df2.drop_duplicates()

print("dropped duplicates !!!")

# Set Up the CRT Code

In [ ]:
# Load the CRT histograms and make some geometry
WEST_X = 383
EAST_X = -383
SOUTH_Z = -179.05 
NORTH_Z = 779.85
FLAT_Y = -378.8
TOPLOW_Y = 632
TOPHIGH_Y = 767


In [ ]:
# implement the TPC Geometry 

NORTH_TPC_Z = 500
SOUTH_TPC_Z = 0
EAST_TPC_X = -200
WEST_TPC_X = 200
TOP_TPC_Y = 200
BOTT_TPC_Y = -200


def cross_top_tpc(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    x_int = (TOP_TPC_Y - bxy)/mxy
    mzy = dir[1]/dir[2]
    bzy = start[1] - mzy*start[2]
    z_int = (TOP_TPC_Y - bzy)/mzy
    if (-200 < x_int < 200) and (0 < z_int < 500):
        return 1
    else:
        return 0

def cross_bottom_tpc(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    x_int = (BOTT_TPC_Y - bxy)/mxy
    mzy = dir[1]/dir[2]
    bzy = start[1] - mzy*start[2]
    z_int = (BOTT_TPC_Y - bzy)/mzy
    if (-200 < x_int < 200) and (0 < z_int < 500):
        return 1
    else:
        return 0


def cross_south_tpc(start, dir):
    mxz = dir[0]/dir[2]
    bxz = start[0] - mxz*start[2]
    x_int = mxz*SOUTH_TPC_Z + bxz
    myz = dir[1]/dir[2]
    byz = start[1] - myz*start[2]
    y_int = myz*SOUTH_TPC_Z + byz
    if (-200 < x_int < 200) and (-200 < y_int < 200):
        return 1
    else:
        return 0
        
def cross_north_tpc(start, dir):
    mxz = dir[0]/dir[2]
    bxz = start[0] - mxz*start[2]
    x_int = mxz*NORTH_TPC_Z + bxz
    myz = dir[1]/dir[2]
    byz = start[1] - myz*start[2]
    y_int = myz*NORTH_TPC_Z + byz
    if (-200 < x_int < 200) and (-200 < y_int < 200):
        return 1
    else:
        return 0


def cross_east_tpc(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    y_int = mxy*EAST_TPC_X + bxy
    mxz = dir[0]/dir[2]
    bxz = start[0] - mxz*start[2]
    z_int = (EAST_TPC_X - bxz)/mxz
    if (-200 < y_int < 200) and (0 < z_int < 500):
        return 1
    else:
        return 0

def cross_west_tpc(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    y_int = mxy*WEST_TPC_X + bxy
    mxz = dir[0]/dir[2]
    bxz = start[0] - mxz*start[2]
    z_int = (WEST_TPC_X - bxz)/mxz
    if (-200 < y_int < 200) and (0 < z_int < 500):
        return 1
    else:
        return 0



def cross_tpc(start, dir):
    top = cross_top_tpc(start, dir)
    bott = cross_bottom_tpc(start, dir)
    north = cross_north_tpc(start, dir)
    south = cross_south_tpc(start, dir)
    east = cross_east_tpc(start, dir)
    west = cross_west_tpc(start, dir)
    if top or bott or north or south or east or west:
        return 1
    else:
        return 0
        

In [ ]:
# Load the CRT histograms

crt_hist_file = ROOT.TFile.Open("/Users/alexanderantonakis/Desktop/crt_heatmaps.root", "READ")

h_tophigh = crt_hist_file.Get("h_tophigh_strip")
h_toplow = crt_hist_file.Get("h_toplow_strip")
h_north = crt_hist_file.Get("h_north_strip")
h_south = crt_hist_file.Get("h_south_strip")
h_east = crt_hist_file.Get("h_east_strip")
h_west = crt_hist_file.Get("h_west_strip")
h_flat = crt_hist_file.Get("h_flat_strip")

c = ROOT.TCanvas("c", "c", 700, 500)
h_flat.Draw("Colz")
c.Draw()

In [ ]:
# Implement the CRT geometry

def cross_crt_tophigh(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    x_int = (TOPHIGH_Y - bxy)/mxy
    mzy = dir[1]/dir[2]
    bzy = start[1] - mzy*start[2]
    z_int = (TOPHIGH_Y - bzy)/mzy
    binx = h_tophigh.GetXaxis().FindBin(z_int)
    biny = h_tophigh.GetYaxis().FindBin(x_int)
    if h_tophigh.GetBinContent(binx, biny) > 0:
        return 1
    else:
        return 0


def cross_crt_toplow(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    x_int = (TOPLOW_Y - bxy)/mxy
    mzy = dir[1]/dir[2]
    bzy = start[1] - mzy*start[2]
    z_int = (TOPLOW_Y - bzy)/mzy
    binx = h_toplow.GetXaxis().FindBin(z_int)
    biny = h_toplow.GetYaxis().FindBin(x_int)
    if h_toplow.GetBinContent(binx, biny) > 0:
        return 1
    else:
        return 0

def cross_crt_south(start, dir):
    mxz = dir[0]/dir[2]
    bxz = start[0] - mxz*start[2]
    x_int = mxz*SOUTH_Z + bxz
    myz = dir[1]/dir[2]
    byz = start[1] - myz*start[2]
    y_int = myz*SOUTH_Z + byz
    binx = h_south.GetXaxis().FindBin(x_int)
    biny = h_south.GetYaxis().FindBin(y_int)
    if h_south.GetBinContent(binx, biny) > 0:
        return 1
    else:
        return 0


def cross_crt_north(start, dir):
    mxz = dir[0]/dir[2]
    bxz = start[0] - mxz*start[2]
    x_int = mxz*NORTH_Z + bxz
    myz = dir[1]/dir[2]
    byz = start[1] - myz*start[2]
    y_int = myz*NORTH_Z + byz
    binx = h_north.GetXaxis().FindBin(x_int)
    biny = h_north.GetYaxis().FindBin(y_int)
    if h_north.GetBinContent(binx, biny) > 0:
        return 1
    else:
        return 0

def cross_crt_east(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    y_int = mxy*EAST_X + bxy
    mxz = dir[0]/dir[2]
    bxz = start[0] - mxz*start[2]
    z_int = (EAST_X - bxz)/mxz
    binx = h_east.GetXaxis().FindBin(z_int)
    biny = h_east.GetYaxis().FindBin(y_int)
    if h_east.GetBinContent(binx, biny) > 0:
        return 1
    else:
        return 0


def cross_crt_west(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    y_int = mxy*WEST_X + bxy
    mxz = dir[0]/dir[2]
    bxz = start[0] - mxz*start[2]
    z_int = (WEST_X - bxz)/mxz
    binx = h_west.GetXaxis().FindBin(z_int)
    biny = h_west.GetYaxis().FindBin(y_int)
    if h_west.GetBinContent(binx, biny) > 0:
        return 1
    else:
        return 0


def cross_crt_flat(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    x_int = (FLAT_Y - bxy)/mxy
    mzy = dir[1]/dir[2]
    bzy = start[1] - mzy*start[2]
    z_int = (FLAT_Y - bzy)/mzy
    binx = h_flat.GetXaxis().FindBin(z_int)
    biny = h_flat.GetYaxis().FindBin(x_int)
    
    if h_flat.GetBinContent(binx, biny) > 0:
        return 1
    else:
        return 0


def cross_crt(start, dir):
    c_tophigh = cross_crt_tophigh(start, dir)
    c_toplow = cross_crt_toplow(start, dir)
    c_north = cross_crt_north(start, dir)
    c_south = cross_crt_south(start, dir)
    c_east = cross_crt_east(start, dir)
    c_west = cross_crt_west(start, dir)
    c_flat = cross_crt_flat(start, dir)
    num_c = c_tophigh + c_toplow + c_north + c_south + c_east + c_west + c_flat
    return num_c
    
print("made crt functions")

In [ ]:
def inTPC_point(point):
    if (-200 <= point[0] <= 200) and (-200 <= point[1] <= 200) and (0 <= point[2] <= 500):
        return 1
    else:
        return 0

def isTPC_Contained(row):
    s = [row["start_x"], row["start_y"], row["start_z"]]
    e = [row["end_x"], row["end_y"], row["end_z"]]
    if inTPC_point(s) and inTPC_point(e):
        return 1
    else:
        return 0
        
def isTrackTPC(row):
    x, y, z = row["start_x"], row["start_y"], row["start_z"]
    if math.isnan(x) or math.isnan(y) or math.isnan(z):
        return 0
    x, y, z = row["end_x"], row["end_y"], row["end_z"]
    if math.isnan(x) or math.isnan(y) or math.isnan(z):
        return 0
    else:
        return 1

trk_av_df1["isTrackTPC"] = trk_av_df1.apply(isTrackTPC, axis=1)
trk_dirt_df1["isTrackTPC"] = trk_dirt_df1.apply(isTrackTPC, axis=1)
trk_av_df1[:5]

In [ ]:
trk_dirt_df1[:5]

In [ ]:

# Select only tracks that have TPC reco

trk_av_df1 = trk_av_df1.query("isTrackTPC == 1")
trk_dirt_df1 = trk_dirt_df1.query("isTrackTPC == 1")

trk_av_df1[:4]

In [ ]:
def N_CRT_AV(row):
    top = row["TOP"]
    sx, sy, sz = row["start_x"], row["start_y"], row["start_z"]
    ex, ey, ez = row["end_x"], row["end_y"], row["end_z"]

    start = [sx, sy, sz]
    end = [ex, ey, ez]

    delx = ex - sx
    dely = ey - sy
    delz = ez - sz
    norm = (delx**2 + dely**2 + delz**2)**0.5
    if norm == 0.0:
        norm = 1.0
    dir = np.array([delx/norm, dely/norm, delz/norm])
    
    c_tophigh = cross_crt_tophigh(start, dir)
    c_toplow = cross_crt_toplow(start, dir)
    c_north = cross_crt_north(start, dir)
    c_south = cross_crt_south(start, dir)
    c_east = cross_crt_east(start, dir)
    c_west = cross_crt_west(start, dir)
    c_flat = cross_crt_flat(start, dir)
    
    if top < 5.8:
        # We have a neutrino
        if inTPC_point(end):
            return -1
        else:
            # we have an exiting track
            N_crt = 0
            # check up
            if dely > 0: 
                N_crt += c_tophigh
                N_crt += c_toplow
                
            else:
                N_crt += c_flat

            if delz > 0:
                N_crt += c_north
            else:
                N_crt += c_south
                
            if delx > 0:
                N_crt += c_west
            else:
                N_crt += c_east
                
        return N_crt
        
    else:
        # We have DIRT
        return -1
        
trk_av_df1["N_CRT_AV"] = trk_av_df1.apply(N_CRT_AV, axis=1)
trk_av_df1[:2]

In [ ]:
def hit_in_crt_toplow(x, y, z):
    dy = 5
    if TOPLOW_Y - dy <= y <= TOPLOW_Y + dy:
        binz = h_toplow.GetXaxis().FindBin(z)
        binx = h_toplow.GetYaxis().FindBin(x)
        if h_toplow.GetBinContent(binz, binx) > 0:
            return 1
    return 0

def hit_in_crt_tophigh(x, y, z):
    dy = 5
    if TOPHIGH_Y - dy <= y <= TOPHIGH_Y + dy:
        binz = h_tophigh.GetXaxis().FindBin(z)
        binx = h_tophigh.GetYaxis().FindBin(x)
        if h_tophigh.GetBinContent(binz, binx) > 0:
            return 1
    return 0

def hit_in_crt_south(x, y, z):
    dz = 5
    if SOUTH_Z - dz <= z <= SOUTH_Z + dz:
        binx = h_south.GetXaxis().FindBin(x)
        biny = h_south.GetYaxis().FindBin(y)
        if h_south.GetBinContent(binx, biny) > 0:
            return 1
    return 0

def hit_in_crt_north(x, y, z):
    dz = 10
    if NORTH_Z - dz <= z <= NORTH_Z + dz:
        binx = h_north.GetXaxis().FindBin(x)
        biny = h_north.GetYaxis().FindBin(y)
        if h_north.GetBinContent(binx, biny) > 0:
            return 1
    return 0

def hit_in_crt_east(x, y, z):
    dx = 5
    if EAST_X - dx <= x <= EAST_X + dx:
        binz = h_east.GetXaxis().FindBin(z)
        biny = h_east.GetYaxis().FindBin(y)
        if h_east.GetBinContent(binz, biny) > 0:
            return 1
    return 0

def hit_in_crt_west(x, y, z):
    dx = 5
    if WEST_X - dx <= x <= WEST_X + dx:
        binz = h_west.GetXaxis().FindBin(z)
        biny = h_west.GetYaxis().FindBin(y)
        if h_west.GetBinContent(binz, biny) > 0:
            return 1
    return 0

def hit_in_crt_flat(x, y, z):
    dy = 10
    if FLAT_Y - dy <= y <= FLAT_Y + dy:
        binz = h_flat.GetXaxis().FindBin(z)
        binx = h_flat.GetYaxis().FindBin(x)
        if h_flat.GetBinContent(binz, binx) > 0:
            return 1
    return 0

def hit_in_crt(x, y, z):
    N1 = hit_in_crt_tophigh(x, y, z) + hit_in_crt_toplow(x, y, z)
    N2 = hit_in_crt_north(x, y, z) + hit_in_crt_south(x, y, z)
    N3 = hit_in_crt_east(x, y, z) + hit_in_crt_west(x, y, z)
    N = N1 + N2 + N3 + hit_in_crt_flat(x, y, z)
    if N > 0:
        return 1
    else:
        return 0

print("made crt hit functions")
    

In [ ]:
print(hit_in_crt(100, TOPHIGH_Y + 100, 200))
print(hit_in_crt(100, TOPHIGH_Y, 2000))

In [ ]:
crt_df[:3]

In [ ]:
crt_df["t"] /= 1000
plt.hist(crt_df.query("t < 0")["t"].values, bins=50)
plt.yscale("log")
plt.show()

plt.hist(crt_df.query("0 <= t <= 10")["t"].values, bins=50)
plt.xlabel("CRT Spacepoint Time [us]", fontsize=14)
plt.yscale("log")
plt.show()

In [ ]:
# Cut CRT hits by beam time

def spInTime(row):
    t = row["t"]
    if 0 <= t <= 2.1:
        return 1
    return 0

crt_df["inTime"] = crt_df.apply(spInTime, axis=1)
crt_df = crt_df.query("inTime == 1")
crt_df[:2]

In [ ]:
def isHit(row):
    x, y, z = row["x"], row["y"], row["z"]
    
    if hit_in_crt(x, y, z):
        return 1
    return 0


crt_df["isHit"] = crt_df.apply(isHit, axis=1)
crt_df[:2]

# Choose if the CRT Hits must be in the actual CRT SP Geometry

In [ ]:
#crt_df = crt_df.query("isHit == 1")
#crt_df[:2]

# AV Event Loop

In [ ]:

# Event Loop Section

# First define the denominator of our event level efficiency
N_AV = 0
#N_AV_C_NU = 0 # fully contained
#N_Dirt = 0
N_AV_H = 0
N_AV_T = 0

N_AV_C = 0
N_AV_C_H = 0
N_AV_C_T = 0

N_AV_H_MC = 0
N_C_H_MC = 0
# Next, we can define some numerators

# Loop over events
for num in range(event_df.shape[0]):
    if num % 10000 == 0:
        print("Analyzing event", num)
        
    row = event_df.iloc[num]
    r, sr, e = row["run"], row["subrun"], row["evt"]
    
    m1 = (slc_av_reco_df["run"].values == r)
    m2 = (slc_av_reco_df["subrun"].values == sr)
    m3 = (slc_av_reco_df["evt"].values == e)
    m4 = (slc_av_reco_df["fmatch_time"].values <= 2.1)
    m5 = (slc_av_reco_df["fmatch_time"].values >= 0)
    m = m1 & m2 & m3 & m4 & m5

    mc1 = (crt_df["run"].values == r)
    mc2 = (crt_df["subrun"].values == sr)
    mc3 = (crt_df["evt"].values == e)    
    mc = mc1 & mc2 & mc3
    N_CRT_MC = np.sum(mc)
    
    AV_slc_ids = np.array(slc_av_reco_df["slc"].values)[m]
    if len(AV_slc_ids) > 0:
        N_AV += 1
        
        #print("AV Slice IDs")
        #print(AV_slc_ids)

        N_CRT = 0
        N_AV_C_NU = 0
        for s in AV_slc_ids:
            m1 = (trk_av_df1["run"].values == r)
            m2 = (trk_av_df1["subrun"].values == sr)
            m3 = (trk_av_df1["evt"].values == e) 
            m4 = (trk_av_df1["slc"].values == s)
            m = m1 & m2 & m3 & m4
            crt_hits = np.array(trk_av_df1["N_CRT_AV"].values)[m]
            n_cont = 0
            for hit in crt_hits:
                if hit < 0:
                    n_cont += 1
                    continue
                else:
                    N_CRT += hit
                    
            if n_cont == len(crt_hits):
                N_AV_C_NU += 1
            
        #print("N_CRT Calculated", N_CRT)
        #print("N_CRT MC", N_CRT_MC)

        #if num > 5:
        #    break
            
        #if num % 1000 == 0:
        #    print("Number of CRT Hits", N_CRT)
            

        if N_AV_C_NU > 0:
            N_AV_C += 1
            if N_CRT == 1:
                N_AV_C_H += 1
            if N_CRT > 1:
                N_AV_C_T += 1
                
            if N_CRT_MC > 0:
                N_C_H_MC += 1
            
        if N_CRT == 1:
            N_AV_H += 1
            
        if N_CRT > 1:
            N_AV_T += 1

        if N_CRT_MC > 0:
            N_AV_H_MC += 1
      

print("Finished Event Loop")

In [ ]:
print("AV", N_AV)
print("N_AV_C", N_AV_C)
print("N_AV_C_H", N_AV_C_H)
print("N_AV_C_T", N_AV_C_T)
print("N_C_H_MC", N_C_H_MC)

In [ ]:
print("AV", N_AV)
print("AV_H", N_AV_H)
print("AV_T", N_AV_T)
print("AV_H_MC", N_AV_H_MC)

In [ ]:
plt.errorbar([1], [N_AV], xerr=[1], yerr=[N_AV**0.5], c="black", fmt="o", linewidth=2, label="AV Events")
plt.errorbar([1], [N_AV_C], xerr=[1], yerr=[N_AV_C**0.5], fmt="o", linewidth=2, label="AV Fully Contained")
plt.errorbar([1], [N_AV_H], xerr=[1], yerr=[N_AV_H**0.5], fmt="o", linewidth=2, label="N CRT = 1")
plt.errorbar([1], [N_AV_T], xerr=[1], yerr=[N_AV_T**0.5], fmt="o", linewidth=2, label="N CRT > 1")
plt.xticks([])
plt.ylabel("Event Count/"+str(TOT_POT)+" POT", fontsize=14)
plt.title("Beam Window Selection Only", fontsize=20)
plt.legend(loc="center")
plt.show()

print("N_AV", N_AV)
print("N_AV Cont", N_AV_C)
print("N_AV_H = 1", N_AV_H)
print("N_AV_T > 1", N_AV_T)


In [ ]:
r_h = N_AV_H/N_AV
r_h_err = r_h*(((1.0/N_AV_H) + (1.0/N_AV))**0.5)

r_t = N_AV_T/N_AV
r_t_err = r_t*(((1.0/N_AV_T) + (1.0/N_AV)))**0.5

N_AV_NOT_C = (N_AV - N_AV_C)

r_h_c = N_AV_H/N_AV_NOT_C
r_h_c_err = r_h_c*(((1.0/N_AV_H) + (1.0/N_AV_NOT_C))**0.5)

r_t_c = N_AV_T/N_AV_NOT_C
r_t_c_err = r_t_c*(((1.0/N_AV_T) + (1.0/N_AV_NOT_C))**0.5)


plt.errorbar([1], [r_h_c], xerr=[1], yerr=[r_h_c_err], fmt="o", linewidth=2, label="N CRT = 1 / Not Contained")
plt.errorbar([1], [r_t_c], xerr=[1], yerr=[r_t_c_err], fmt="o", linewidth=2, label="N CRT > 1 / Not Contained")
plt.errorbar([1], [r_h], xerr=[1], yerr=[r_h_err], fmt="o", linewidth=2, label="N CRT = 1 / All")
plt.errorbar([1], [r_t], xerr=[1], yerr=[r_t_err], fmt="o", linewidth=2, label="N CRT > 1 / All")

plt.xticks([])
plt.ylabel("Event Fraction/"+str(TOT_POT)+" POT", fontsize=14)
plt.title("Beam Window Selection Only", fontsize=20)
plt.legend(loc="center")
plt.ylim([0, 1])
plt.show()

# Dirt Study

In [ ]:
# Need to add the slc true vertex to reco tracks

trk_dirt_df1[:4]

In [ ]:
# Add vertex to dirt tracks
df_x = slc_dirt_df[['run', 'subrun', 'evt', 'slc', 'vtx_x']]
df_y = slc_dirt_df[['run', 'subrun', 'evt', 'slc', 'vtx_y']]
df_z = slc_dirt_df[['run', 'subrun', 'evt', 'slc', 'vtx_z']]

trk_dirt_df1 = trk_dirt_df1.merge(df_x, on=['run', 'subrun', 'evt', 'slc'], how='left')
trk_dirt_df1 = trk_dirt_df1.merge(df_y, on=['run', 'subrun', 'evt', 'slc'], how='left')
trk_dirt_df1 = trk_dirt_df1.merge(df_z, on=['run', 'subrun', 'evt', 'slc'], how='left')
trk_dirt_df1[:5]

In [ ]:
def N_CRT_DIRT(row):
    top = row["TOP"]
    vx, vy, vz = row["vtx_x"], row["vtx_y"], row["vtx_z"]
    sx, sy, sz = row["start_x"], row["start_y"], row["start_z"]
    ex, ey, ez = row["end_x"], row["end_y"], row["end_z"]

    vtx = [vx, vy, vz]
    start = [sx, sy, sz]
    end = [ex, ey, ez]

    delx1 = sx - vx
    dely1 = sy - vy
    delz1 = sz - vz
    norm1 = (delx1**2 + dely1**2 + delz1**2)**0.5
    if norm1 == 0.0:
        norm1 = 1.0
    dir1 = np.array([delx1/norm1, dely1/norm1, delz1/norm1])

    delx2 = ex - sx
    dely2 = ey - sy
    delz2 = ez - sz
    norm2 = (delx2**2 + dely2**2 + delz2**2)**0.5
    if norm2 == 0.0:
        norm2 = 1.0
    dir2 = np.array([delx2/norm2, dely2/norm2, delz2/norm2])


    S_MIN_Y = h_south.GetYaxis().GetXmin()
    S_MAX_Y = h_south.GetYaxis().GetXmax()
            
    c_tophigh1 = cross_crt_tophigh(vtx, dir1)
    c_toplow1 = cross_crt_toplow(vtx, dir1)
    c_north1 = cross_crt_north(vtx, dir1)
    c_south1 = cross_crt_south(vtx, dir1)
    c_east1 = cross_crt_east(vtx, dir1)
    c_west1 = cross_crt_west(vtx, dir1)
    c_flat1 = cross_crt_flat(vtx, dir1)

    c_tophigh2 = cross_crt_tophigh(end, dir2)
    c_toplow2 = cross_crt_toplow(end, dir2)
    c_north2 = cross_crt_north(end, dir2)
    c_south2 = cross_crt_south(end, dir2)
    c_east2 = cross_crt_east(end, dir2)
    c_west2 = cross_crt_west(end, dir2)
    c_flat2 = cross_crt_flat(end, dir2)

    N_crt = 0
    # check up
    if vy > TOPHIGH_Y:     
        N_crt = c_tophigh1 + c_toplow1 + c_north1 + c_south1 + c_east1 + c_west1
            
    elif vy > TOPLOW_Y:     
        N_crt = c_toplow1 + c_north1 + c_south1 + c_east1 + c_west1 + c_flat1            
                
    elif S_MIN_Y < vy < S_MAX_Y:
        if vz < SOUTH_Z:
            N_crt += c_south1
        if vz > NORTH_Z:
            N_crt += c_north1
        if vx > WEST_X:
            N_crt += c_west1
        if vx < EAST_X:
            N_crt += c_east1
        
    elif vy < FLAT_Y:
        N_crt += c_flat1
        if vz < SOUTH_Z:
            N_crt += c_south1
        if vz > NORTH_Z:
            N_crt += c_north1
        if vx > WEST_X:
            N_crt += c_west1
        if vx < EAST_X:
            N_crt += c_east1
                
    # Next, check if the endpoint is contained
       
    if inTPC_point(end) == 0:
        if dir2[2] > 0:
            N_crt += c_north2
        if dir2[2] < 0:
            N_crt += c_south2
        if dir2[1] > 0:
            N_crt += c_toplow2
            N_crt += c_tophigh2
        if dir2[1] < 0:
            N_crt += c_flat2
        if dir2[0] > 0:
            N_crt += c_west2
        if dir2[0] < 0:
            N_crt += c_east2
    return N_crt

    
trk_dirt_df1["N_CRT_DIRT"] = trk_dirt_df1.apply(N_CRT_DIRT, axis=1)
trk_dirt_df1[:5]

# Dirt Event Loop

In [ ]:

# Event Loop Section

# First define the denominator of our event level efficiency
N_DIRT = 0

#N_Dirt = 0
N_DIRT_H = 0
N_DIRT_T = 0

N_DIRT_H_MC = 0

# Next, we can define some numerators

# Loop over events
for num in range(event_df.shape[0]):
    if num % 10000 == 0:
        print("Analyzing event", num)
        
    row = event_df.iloc[num]
    r, sr, e = row["run"], row["subrun"], row["evt"]
    
    m1 = (slc_dirt_reco_df["run"].values == r)
    m2 = (slc_dirt_reco_df["subrun"].values == sr)
    m3 = (slc_dirt_reco_df["evt"].values == e)
    m4 = (slc_dirt_reco_df["fmatch_time"].values <= 2.1)
    m5 = (slc_dirt_reco_df["fmatch_time"].values >= 0)
    m = m1 & m2 & m3 & m4 & m5

    mc1 = (crt_df["run"].values == r)
    mc2 = (crt_df["subrun"].values == sr)
    mc3 = (crt_df["evt"].values == e)    
    mc = mc1 & mc2 & mc3
    N_CRT_MC = np.sum(mc)
 
    DIRT_slc_ids = np.array(slc_dirt_reco_df["slc"].values)[m]
    if len(DIRT_slc_ids) > 0:
        N_DIRT += 1
        
        #print("AV Slice IDs")
        #print(AV_slc_ids)

        N_CRT = 0
        for s in DIRT_slc_ids:
            m1 = (trk_dirt_df1["run"].values == r)
            m2 = (trk_dirt_df1["subrun"].values == sr)
            m3 = (trk_dirt_df1["evt"].values == e) 
            m4 = (trk_dirt_df1["slc"].values == s)
            m = m1 & m2 & m3 & m4
            crt_hits = np.sum(np.array(trk_dirt_df1["N_CRT_DIRT"].values)[m])
            N_CRT += crt_hits
            
            #N_CRT += crt_hits

        #if num % 1000 == 0:
        #    print("Number of CRT Hits", N_CRT)    
            
        if N_CRT == 1:
            N_DIRT_H += 1
            
        if N_CRT > 1:
            N_DIRT_T += 1

        if N_CRT_MC > 0:
            N_DIRT_H_MC += 1
    
    #print(slc_tops)        
    #m1 = (tops != 7) # not cosmic
    #m2 = (tops != 6) # not dirt
    #m = m1 & m2
    #n = sum(m)
    
    #if num > 11:
    #    break


print("Finished Event Loop")

In [ ]:
print("N_DIRT", N_DIRT)
print("N_DIRT_H", N_DIRT_H)
print("N_DIRT_T", N_DIRT_T)
print("N_DIRT_H_MC", N_DIRT_H_MC)

In [ ]:
plt.errorbar([1], [N_DIRT], xerr=[1], yerr=[N_DIRT**0.5], c="black", fmt="o", linewidth=2, label="DIRT Events")
plt.errorbar([1], [N_DIRT_H], xerr=[1], yerr=[N_DIRT_H**0.5], fmt="o", linewidth=2, label="N CRT = 1")
plt.errorbar([1], [N_DIRT_T], xerr=[1], yerr=[N_DIRT_T**0.5], fmt="o", linewidth=2, label="N CRT > 1")
plt.xticks([])
plt.ylabel("Event Count/"+str(TOT_POT)+" POT", fontsize=14)
plt.title("Beam Window Selection Only", fontsize=20)
plt.legend(loc="center")
plt.show()

print("N_DIRT", N_DIRT)
print("N_DIRT_H", N_DIRT_H)
print("N_DIRT_T", N_DIRT_T)

In [ ]:
r_h = N_DIRT_H/N_DIRT
r_h_err = r_h*(((1.0/N_DIRT_H) + (1.0/N_DIRT))**0.5)

r_t = N_DIRT_T/N_DIRT
r_t_err = r_t*(((1.0/N_DIRT_T) + (1.0/N_DIRT)))**0.5


plt.errorbar([1], [r_h], xerr=[1], yerr=[r_h_err], fmt="o", linewidth=2, label="N CRT = 1 / All DIRT")
plt.errorbar([1], [r_t], xerr=[1], yerr=[r_t_err], fmt="o", linewidth=2, label="N CRT > 1 / All DIRT")

plt.xticks([])
plt.ylabel("Event Fraction/"+str(TOT_POT)+" POT", fontsize=14)
plt.title("Beam Window Selection Only", fontsize=20)
plt.legend(loc="center")
plt.ylim([0, 1])
plt.show()

# Check that the CRT Event Number Matches the Slice Event Number

In [ ]:
runs = crt_df["run"].values
subruns = crt_df["subrun"].values
evts = crt_df["evt"].values

data = {'run': runs,
        'subrun': subruns,
        'evt': evts}

crt_event_df = pd.DataFrame(data)

crt_event_df = crt_event_df.drop_duplicates()

# Drop duplicates to find unique combinations
#unique_combinations = event_df.drop_duplicates()

# Get the number of unique combinations
#N_EVENTS = event_df.shape[0]

print("N_EVENTS Slice:", event_df.shape[0])
print("N_EVENTS CRT:", crt_event_df.shape[0])

# Combined Event Loop

In [ ]:

# Event Loop Section

# First define the denominator of our event level efficiency
N_AV = 0
N_AV_H = 0
N_AV_T = 0
N_AV_H_MC = 0


N_AV_C = 0
N_AV_C_H = 0
N_AV_C_T = 0
N_AV_H_MC = 0

N_DIRT = 0
N_DIRT_H = 0
N_DIRT_T = 0
N_DIRT_H_MC = 0


# Loop over events
for num in range(event_df.shape[0]):
    if num % 10000 == 0:
        print("Analyzing event", num)
        
    row = event_df.iloc[num]
    r, sr, e = row["run"], row["subrun"], row["evt"]

    # AV Selection
    m1av = (slc_av_reco_df["run"].values == r)
    m2av = (slc_av_reco_df["subrun"].values == sr)
    m3av = (slc_av_reco_df["evt"].values == e)
    m4av = (slc_av_reco_df["fmatch_time"].values <= 2.1)
    m5av = (slc_av_reco_df["fmatch_time"].values >= 0)
    mav = m1av & m2av & m3av & m4av & m5av

    # Dirt Selection
    m1d = (slc_dirt_reco_df["run"].values == r)
    m2d = (slc_dirt_reco_df["subrun"].values == sr)
    m3d = (slc_dirt_reco_df["evt"].values == e)
    m4d = (slc_dirt_reco_df["fmatch_time"].values <= 2.1)
    m5d = (slc_dirt_reco_df["fmatch_time"].values >= 0)
    md = m1d & m2d & m3d & m4d & m5d

    # Simple CRT MC selection
    mc1 = (crt_df["run"].values == r)
    mc2 = (crt_df["subrun"].values == sr)
    mc3 = (crt_df["evt"].values == e)    
    mc = mc1 & mc2 & mc3
    N_CRT_MC = np.sum(mc)
    
    AV_slc_ids = np.array(slc_av_reco_df["slc"].values)[mav]
    DIRT_slc_ids = np.array(slc_dirt_reco_df["slc"].values)[md]

    if len(AV_slc_ids) > 0:
        N_AV += 1

    if len(DIRT_slc_ids) > 0:
        N_DIRT += 1

    if N_CRT_MC > 0 and len(DIRT_slc_ids) > 0:
        N_DIRT_H_MC += 1

    if N_CRT_MC > 0  and len(AV_slc_ids) > 0:
        N_AV_H_MC += 1
        
        #print("AV Slice IDs")
        #print(AV_slc_ids)

    N_CRT = 0
    for s in AV_slc_ids:
        m1 = (trk_av_df1["run"].values == r)
        m2 = (trk_av_df1["subrun"].values == sr)
        m3 = (trk_av_df1["evt"].values == e) 
        m4 = (trk_av_df1["slc"].values == s)
        m = m1 & m2 & m3 & m4
        crt_hits = np.array(trk_av_df1["N_CRT_AV"].values)[m]
            
        for hit in crt_hits:
            if hit < 0:
                #n_cont += 1
                continue
            else:
                N_CRT += hit
          
    for s in DIRT_slc_ids:
        m1 = (trk_dirt_df1["run"].values == r)
        m2 = (trk_dirt_df1["subrun"].values == sr)
        m3 = (trk_dirt_df1["evt"].values == e) 
        m4 = (trk_dirt_df1["slc"].values == s)
        m = m1 & m2 & m3 & m4
        crt_hits = np.sum(np.array(trk_dirt_df1["N_CRT_DIRT"].values)[m])
        N_CRT += crt_hits
            
    if N_CRT > 0 and len(AV_slc_ids) > 0:
        N_AV_H += 1
        
    if N_CRT > 1 and len(AV_slc_ids) > 0:
        N_AV_T += 1
    
    if N_CRT > 0 and len(DIRT_slc_ids) > 0:
        N_DIRT_H += 1
        
    if N_CRT > 1 and len(DIRT_slc_ids) > 0:
        N_DIRT_T +=1       


print("Finished Event Loop")

In [ ]:
print("N_AV", N_AV)
print("N_AV_H", N_AV_H)
print("N_AV_T", N_AV_T)
print("N_AV_H_MC", N_AV_H_MC)

#N_AV_C
#N_AV_C_H
#N_AV_C_T
#N_AV_H_MC

print("")
print("")

print("N_DIRT", N_DIRT)
print("N_DIRT_H", N_DIRT_H)
print("N_DIRT_T", N_DIRT_T)
print("N_DIRT_H_MC", N_DIRT_H_MC)

In [ ]:
print((1.0*N_AV_H)/N_AV)
print((1.0*N_DIRT_H)/N_DIRT)

In [ ]:
print((1.0*N_AV_T)/N_AV)
print((1.0*N_DIRT_T)/N_DIRT)